In [34]:
import numpy as np
import pandas as pd
from pyproj import Transformer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

In [35]:
cat1 = pd.read_csv('10950_2019_9864_MOESM2_ESM.csv')
cat2 = pd.read_csv('SRL_2017007_esupp_matched_filter_cat_pawnee.csv',delimiter="\t")

In [ ]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
cat1[['x', 'y']] = cat1.apply(
    lambda row: transformer.transform(row['long'], row['lat']), axis=1, result_type='expand'
)
cat2[['x', 'y']] = cat2.apply(
    lambda row: transformer.transform(row['longitude'], row['latitude']), axis=1, result_type='expand'
)

df = cat1
df.drop_duplicates()

,year,month,day,hour,minute,second,lat,long,depth,mag,x,y
0,2008,10,21,0,25,59.13,35.67700,-97.13100,5.000,1.258420,-1.081257e+07,4.256268e+06
1,2008,10,21,2,47,2.45,35.67700,-97.13100,5.000,0.965415,-1.081257e+07,4.256268e+06
2,2008,10,21,5,15,0.11,35.03300,-97.18100,5.000,2.700000,-1.081814e+07,4.168367e+06
3,2008,10,21,5,27,8.38,35.03300,-97.18100,5.000,2.569460,-1.081814e+07,4.168367e+06
4,2008,10,21,5,33,7.22,35.67700,-97.13100,5.000,2.176790,-1.081257e+07,4.256268e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
209403,2016,12,30,21,27,52.05,35.67334,-97.39604,5.717,1.920350,-1.084208e+07,4.255766e+06
209404,2016,12,30,22,0,25.84,36.43382,-96.90550,4.680,1.880900,-1.078747e+07,4.360480e+06
209405,2016,12,30,22,16,41.00,36.43381,-96.90656,3.802,1.600000,-1.078759e+07,4.360478e+06
209406,2016,12,30,22,50,56.50,35.96477,-97.82970,8.179,2.500000,-1.089035e+07,4.295775e+06


In [38]:
# Outlier removal
z_thres = 3.0

z_scores = (df - df.mean())/df.std()

outliers = (z_scores.abs() > z_thres).any(axis=1)

df_cleaned = df[~outliers]